In [1]:
import json
import os
import pickle
import milsed
from sklearn.model_selection import ShuffleSplit
import pandas as pd
from multiprocessing import Process

Using TensorFlow backend.


In [2]:
cd ~/dev/milsed/models/

/home/js7561/dev/milsed/models


In [3]:
# SETUP STUFF
seed = 20170612
OUTPUT_PATH = '/home/js7561/dev/milsed/models/resources/'
pump = pickle.load(open('/home/js7561/dev/milsed/models/resources/pump.pkl', 'rb'))

# GET VALIDATION TRACKS
idx_train_ = pd.read_json(os.path.join(OUTPUT_PATH, 'index_train.json'))
# Split the training data into train and validation
splitter_tv = ShuffleSplit(n_splits=1, test_size=0.25,
                           random_state=seed)
train, val = next(splitter_tv.split(idx_train_))

idx_train = idx_train_.iloc[train]
idx_val = idx_train_.iloc[val]

pumpfolder = '/beegfs/js7561/datasets/dcase2017/task4_official/combined/features_silence/'
labelfile = ('/beegfs/js7561/datasets/dcase2017/task4_official/combined/metadata/labels/'
             '/groundtruth_strong_label_testing_set.csv')
duration = 10.0

In [4]:
val_idx = [str(x) for x in idx_val.id.values]

In [5]:
# val_idx = val_idx[:10]

In [6]:
len(val_idx)

12793

In [7]:
def recompute_results_validation(version, use_orig_duration=True, save=True):
    # Compute results
    weight_path = os.path.join(OUTPUT_PATH, version, 'model.h5')
    params = json.load(open(os.path.join(OUTPUT_PATH, version, 'params.json'), 'r'))
    model, inputs, outputs = milsed.models.MODELS[params['modelname']](pump, alpha=1.0)
    model.load_weights(weight_path)
    
    # normal results
    results = milsed.eval.score_model_validation(
        OUTPUT_PATH, pump, model, val_idx, pumpfolder, labelfile, duration, version, 
        use_tqdm=True, use_orig_duration=use_orig_duration, save_jams=save, weak_from_strong=False)
    
    # weak from strong results
    results_wfs = milsed.eval.score_model_validation(
        OUTPUT_PATH, pump, model, val_idx, pumpfolder, labelfile, duration, version, 
        use_tqdm=True, use_orig_duration=use_orig_duration, save_jams=save, weak_from_strong=True)
    
    # Save them to disk
    if save:
        resultsfile = os.path.join(OUTPUT_PATH, version, 'predictions_validation', 'predictions', 'results.json')
        with open(resultsfile, 'w') as fp:
            json.dump(results, fp, indent=2)
            
        resultsfile_wfs = os.path.join(OUTPUT_PATH, version, 'predictions_validation', 
                                       'predictions_weakfromstrong', 'results.json')
        with open(resultsfile_wfs, 'w') as fp:
            json.dump(results_wfs, fp, indent=2)
    
    print('Completed {}'.format(version))
    return results, results_wfs

In [8]:
models = []
for i in range(23, 24):
    models.append('model0{}'.format(i))
    
for n, version in enumerate(models):
    print('Launched {}'.format(version))
    recompute_results_validation(version, save=True, use_orig_duration=True)

Launched model023


Evaluating the model: 100%|██████████| 12793/12793 [30:48<00:00,  8.50it/s]
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
Evaluating the model: 100%|██████████| 12793/12793 [26:05<00:00,  7.06it/s]


Completed model023
